In [ ]:
#makes a recommendation function based on my listen time to each song and that artist's spotify popularity

In [6]:
#get relevant libraries
import json
import pandas as pd
from datetime import datetime
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from googlesearch import search  
import re
import seaborn as sns
import math
import random
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
#fuzzy needs to be added to the library
#open terminal and paste: conda install -c conda-forge fuzzywuzzy
from fuzzywuzzy import process
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

C:\Users\Tara\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
#set spotify credentials and generate spotify object
os.environ['SPOTIPY_CLIENT_ID']='CLIENT_ID_HERE'
os.environ['SPOTIPY_CLIENT_SECRET']='CLIENT_SECRET_Here'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [95]:
#read in streaming data, combine into one df, and get list of all unique artists listened to
df=pd.read_json(r'StreamingHistory0.json')
df2=pd.read_json(r'StreamingHistory1.json')
df=pd.concat([df,df2])
df=df.reset_index()
df_music=df.loc[(df['artistName']!='Overdue')&(df['artistName']!='Sinisterhood')]
df_music_played=df_music.loc[df_music['msPlayed']!=0]
artistlist=df_music_played['artistName'].drop_duplicates()
artistlist=artistlist.reset_index()['artistName']

In [96]:
#use total msPlayed for each song as a proxy for rating
df_music_ratings=df_music_played.groupby(['artistName','trackName']).agg({'msPlayed':'sum'}).reset_index().sort_values('msPlayed')
df_music_ratings['MyRating']=df_music_ratings['msPlayed']*100/max(df_music_ratings['msPlayed'])

In [59]:
#search for track by artist name and song name

#spotify.search(str(df_music_ratings['artistName'][0]+' '+df_music_ratings['trackName'][0]),1,0,'track')['tracks']['items'][0]['uri']

'spotify:track:4bVuIlGQBMWS7vIhcx8Ae4'

In [82]:
music_ratings_artists=df_music_ratings[['artistName']].drop_duplicates()
music_ratings_artists['Popularity']=''
music_ratings_artists=music_ratings_artists.reset_index()

In [ ]:
for i in range(len(music_ratings_artists)):
    print(i)
    music_ratings_artists.loc[i,'Popularity']=spotify.search(music_ratings_artists.loc[i,'artistName'],1,0,'artist')['artists']['items'][0]['popularity']
music_ratings_artists.to_csv('MyArtistsWPopularity.csv')

In [97]:
df_music_ratings=pd.merge(df_music_ratings,music_ratings_artists,how='left',on='artistName')
df_music_ratings['artistName / trackName']=df_music_ratings['artistName']+' / '+df_music_ratings['trackName']
df_music_ratings=df_music_ratings[['artistName / trackName','MyRating','Popularity']]

In [98]:
df_me=df_music_ratings[['artistName / trackName','MyRating']].rename(columns={'MyRating':'Rating'})
df_me['User']='Tara'
df_sp=df_music_ratings[['artistName / trackName','Popularity']].rename(columns={'Popularity':'Rating'})
df_sp['User']='GP'
df_music_ratings=pd.concat([df_me,df_sp])

In [99]:
df_music_backup=df_music_ratings.copy()

In [100]:
#Combine datasets into Sparse Matrix
df_music_ratings = df_music_ratings.pivot(index = 'artistName / trackName', columns='User',values = 'Rating').fillna(0)
matrix_music_users = csr_matrix(df_music_ratings.values)
matrix_music_users

<3457x2 sparse matrix of type '<class 'numpy.float64'>'
	with 6907 stored elements in Compressed Sparse Row format>

In [103]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 10)
model_knn.fit(matrix_music_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [162]:
#writes the recommender function
def recommender(song_name, data, model, n_recommendations):
    model.fit(data)
    idx = process.extractOne(song_name, df_music_backup['artistName / trackName'])[2]
    #print('Song Selected: ', df_music_backup['artistName / trackName'][idx].reset_index()['artistName / trackName'][0], 'Index: ',idx)
    print('Song Selected: ', df_music_backup['artistName / trackName'][idx].reset_index()['artistName / trackName'][0])
    print('Searching for recommendations.....')
    distances, indices = model.kneighbors(data[idx], n_neighbors = n_recommendations)
    for i in indices[0]:
        if i!=idx:
            print(df_music_backup['artistName / trackName'][i].reset_index()['artistName / trackName'][0])

In [165]:
recommender('Lady Gaga / LoveGame',matrix_music_users,model_knn,20)

Song Selected:  Lady Gaga / LoveGame
Searching for recommendations.....
Halsey / Lie
Madge / How to Play
Screamin' Jay Hawkins / Sweet Ginny
Pitbull / Timber (feat. Ke$ha)
Sarah Joy / Rewrite the Stars - Cello Version
Julien Baker / Bite The Hand
Sharon Van Etten / Like I Used To
Bad Suns / Violet
BETWEEN FRIENDS / ok
The Shins / A Comet Appears
Adele / Can I Get It
The Wombats / My First Wedding
Sipper / Txt
ABBA / As Good As New
Kesha / We R Who We R
George Winston / Early Morning Range
WALK THE MOON / Tightrope
I DONT KNOW HOW BUT THEY FOUND ME / Absinthe
Kanye West / Believe What I Say
